## From Scikit-learn to Arduino

Machine learning on Arduino boards is it possible?

Do you want to run the model you trained in Python into any C++ project, be it Arduino, STM32, ESP32?

## Step1 load data
To train a classifier, we need some data. If you're starting from zero and don't have already a preferred folder structure, I suggest you to create a folder that will hold the data you collect.

Inside this folder, create a dedicated file (.csv) for each of the classes you want to classify, putting a sample on each line. If doing so, you can use the next function to load this data.

In [37]:
!mkdir datafolder

mkdir: cannot create directory ‘datafolder’: File exists


In [38]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/sample_binarydata.csv -O /content/datafolder/sample_binarydata.csv

--2023-11-07 13:51:23--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/sample_binarydata.csv
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 52.95.148.158, 52.95.150.142, 52.95.142.62, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|52.95.148.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23777 (23K) [text/csv]
Saving to: ‘/content/datafolder/sample_binarydata.csv’

/content/datafolder 100%[===================>]  23.22K  --.-KB/s    in 0.09s   

2023-11-07 13:51:23 (266 KB/s) - ‘/content/datafolder/sample_binarydata.csv’ saved [23777/23777]



In [47]:
import numpy as np

def load_data(file):
    data = np.loadtxt(file, dtype=float, delimiter=',')
    classmap = {
                0:'survived',
                1:'died',
                }
    return data, classmap

In [40]:
data = np.loadtxt('datafolder/sample_binarydata.csv', dtype=float, delimiter=',')
data

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

## Step2 Train Classifier

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def modeling(data):
    X, y = data[:, :-1], data[:, -1]
    k = 5
    k_best = SelectKBest(score_func=f_classif, k=k)
    k_best.fit(X, y)
    selected_feature_indices = k_best.get_support(indices=True)

    clf = RandomForestClassifier(20, max_depth=10, random_state=667)

    return clf.fit(X[:, selected_feature_indices], y)

## Step3 Export to C
Now you can convert the trained classifier to plain C code using the micromlgen package.



In [42]:
!pip install micromlgen -q

In [43]:
from micromlgen import port

dataset, classmap = load_data('datafolder/sample_binarydata.csv')
classifier = modeling(dataset)
classmap

{0: 'survived', 1: 'died'}

In [44]:
## testing classifier prediction
#classifier.predict([[6.,148.,33.6,0.627,50.]])[0] ## class1.0
classifier.predict([[1., 85.,26.6,0.351,31.]])[0] ##  class0.0

0.0

In [45]:
c_code = port(classifier, classmap=classmap)
print(c_code)

#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class RandomForest {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        uint8_t votes[2] = { 0 };
                        // tree #1
                        if (x[1] <= 154.5) {
                            if (x[4] <= 29.5) {
                                if (x[3] <= 0.6924999952316284) {
                                    if (x[2] <= 56.599998474121094) {
                                        if (x[1] <= 112.5) {
                                            if (x[2] <= 30.899999618530273) {
                                                votes[0] += 1;
                                            }

                                            else {
                                                if (x[1] <= 78.5) {
                 

In [46]:
## write model into model.h
file_path = "model.h"

with open(file_path, 'w') as file:
    file.write(c_code)

## Step4 Load model into Arduino

In [ ]:
# // put the code you got in Step 3 into this file
# #include "model.h"

# // this class will be different if you used another type of classifier, just check the model.h file
# Eloquent::ML::Port::RandomForest classifier;

# void classify() {
#     float x_sample[] = { /* fill this vector with sample values */ };

#     Serial.print("Predicted class: ");
#     Serial.println(classifier.predictLabel(x_sample));
# }